# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [342]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [353]:
def foldl2(f, x0, lst):
    return foldr(..., ..., lst)(...)

In [1]:
def foldr2(f, x0, lst):
    return foldl(..., ..., lst)(...)


In [456]:
[1, 2, 3][:-1]

[1, 2]

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [6]:
# Наивное решение
def check_inv(a: str, b: str) -> bool:
    for begin in range(len(a)-len(b)+1):
        if sorted(a[begin:begin+len(b)]) == sorted(b):
            return True
    return False
check_inv('abcrotm', 'tro')

True

In [449]:
# Оптимальное решение

def make_counter(a: str) -> dict:
    counter = {}
    for char in a:
        counter[char] = counter.get(char, 0)+1
    return counter

def get_distance(a: dict, b:dict):
    dist = 0
    for char in set(list(a.keys())+list(b.keys())):
        dist+=abs(a.get(char, 0) - b.get(char, 0))
    return dist


def check_inv(a: str, b:str) -> bool:
    a_len = len(a)
    b_len = len(b)
    
    a_counter = make_counter(a[:b_len])
    b_counter = make_counter(b)
    distance = get_distance(a_counter, b_counter)
    if distance == 0:
            return True
    
    for begin in range(1, a_len-b_len+1):
        left_char = a[begin-1]
        right_char = a[begin+b_len-1]
        
        old_left = abs(a_counter.get(left_char, 0) - b_counter.get(left_char, 0))
        old_right = abs(a_counter.get(right_char, 0) - b_counter.get(right_char, 0))
        
        a_counter[left_char] = a_counter.get(left_char, 0)-1
        a_counter[right_char] = a_counter.get(right_char, 0)+1
        
        curr_left = abs(a_counter.get(left_char, 0) - b_counter.get(left_char, 0))
        curr_right = abs(a_counter.get(right_char, 0) - b_counter.get(right_char, 0))
        
        distance = distance + (curr_left-old_left) + (curr_right-old_right)
        if distance == 0:
            return True
    return False

check_inv('abcrotm', 'tor')

True

## Оценка сложности алгоритма

### 1) Наивное решение

<b>Идея</b>\
Проходимся по строке <code>a</code> окном длины <code>len(b)</code> и проверяем, равны ли отсортированные строки друг другу

<b>Асимптотика</b>
* Всего шагов окна - $O(len(a))$
* На каждом шаге в худшем случае $O(len(b))$ операций
* Сложность первого алгоритма - $O(len(a)*len(b))$
<br/><br/>


### 2) Оптимальное решение

<b>Идея</b>\
Вначале создается два словаря, содержащих буквы и их количество в строке b и в срезе строки а длины <code>len(b)</code>\
Высчитывается <em>расстояние</em> между двумя словарями, то есть сумма модулей разности значений по каждому символу\
Затем, начиная со 2 символа, проходимся окном длины <code>len(b)</code> по строке <code>a</code>, однако не высчитываем <em>расстояние</em> с нуля, а делаем это по двум символам: удаленному и добавленому.\
Вследствие чего <em>расстояние</em> может уменьшиться на 2, увеличиться на 2, либо остаться прежним.
Если <em>расстояние</em> равно нулю - мы нашли перестановку

<b>Асимптотика</b>
* Создание словарей и рассчет <em>расстояния</em> - $O(len(b))$, но так как эти операции проводятся единожды, а также $len(b) << len(a)$, в итоговой асимптотике они не учитываются
* Всего шагов окна - $O(len(a))$
* Изменение <em>расстояния</em> и проверка равенства нулю - $O(1)$
* Сложность второго алгоритма - <code>len(a)</code>



# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [413]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self, lst=[]):
        if self.left == self.right == None:
            lst.append(self.value)
        if self.left != None:
            iter_tree(self.left, lst)
        if self.right != None:
            iter_tree(self.right, lst)
        return iter(lst)
    
    def __repr__(self):
        if self.right is not None:
            text = '<{}({value!r}, {left!r}, {right!r})>'
        elif self.left is not None:
            text = '<{}({value!r}, {left!r})>'
        else:
            text = '<{}({value!r})>'
        return text.format(type(self).__name__, **vars(self))
    
    def __str__(self):
        return f'В объекте класса Tree листья имеют значения: {list(self.__iter__())}'


In [414]:
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))
list(tree)

[3, 4, 2]

In [415]:
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))
str(tree)

'В объекте класса Tree листья имеют значения: [3, 4, 2, 3, 4, 2]'

In [416]:
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))
tree

<Tree(0, <Tree(1, <Tree(3)>, <Tree(4)>)>, <Tree(2)>)>

# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

## Идея алгоритма

### 1) Разбиение выражения на составные части
Алгоритм будет работать со списоком, в котором находятся числа и знаки математических операций

Однако воспользоваться обычным re.split() мы не можем, так как калькулятор должен уметь работать с отрицательными числами

Для этого, после вызова операции <code>re.split('(\+|\-|\*|\/|\(|\))', expr)</code>, разделяющей строку на блоки с числами и математическими знаками, мы проводим проверку всех знаков '-', перед которыми стоит не число (в противном случае минус является разделяющим, то есть вычитает одно число из другого)

Затем все строки с числами приводятся к типу <code>float</code>, а все числа, перед который стоит неразделяющий минус, домножаются на $(-1)$

### 2) Обработчик скобок
Первым делом выполняются операции, находящиеся в скобках.

Так как скобок внутри может быть сколько угодно, нужно при помощи рекурсии вычислить значение внутри скобок, после чего заменить это выражение на число, равное этому выражению. 

Для этого я написал функцию <code>get_next_bracket_index()</code>, которая принимает выражение, следующее после открытой скобки, и ищет индекс, соответствующий закрывающей скобке для данной открытой. Делается это при помощи счетчика (открывающая скобка добавляет единицу к счетчику, закрывающая отнимает единицу, при равенстве счетчика нулю будет найдена нужная закрывающая скобка)

Затем выражение внутри скобок передается в функцию <code>calc</code> и высчитывается ее значение

### 3) Операции умножения и деления
Блок с обработкой знаков умножения и деления следует уже после того, как в выражении обработаются все скобки. То есть выражение будет иметь следующий вид: числа, разделенные знаками $+, -, *, /$ 

Трудность кроется в том, что операции умножения и деления должны выполняться слева направо, ведь в противном случае
можно столкнуться с ошибкой вида $10*2/5 = 4$ при приоритете выполнения деления и $10/2*5 = 1$ при приоритете выполнения умножения. 

По этой причине данный блок выполняется до тех пор, пока в выражении не останется знаков умножения и деления.

Ищется самый левый индекс, соответствующий одному из данных знаков, после чего производится произведение или деление двух рядом стоящих чисел, а затем выражение $a(*|/)b$ в списке с выражением заменяется на число $c$, равное значению этого выражения

### 4) Операции сложения и вычитания
Данный блок аналогичен предыдущему, однако сделано это было для лаконичности и читаемости кода, ведь в данной задаче операции сложения и вычитания являются ассоциативными. 

Однако при отсутствии проверки отрицательных чисел могли бы возникать ошибки в выражениях типа $-2-2$, так как первая операция вычитания не имела бы числа слева. 

### 5) Заключение
Как можно заметить, идея с разбиением строки на блоки позволяет обрабатывать не только целочисленные выражения, но и числа с плавающей точкой. Также в калькулятор можно с легкостью добавить другие математические знаки (по типу факториала и возведения в степень), ведь блочная структура с использованием рекурсии легко преобразует данные выражения в числовые значения\


In [374]:
import re
from typing import Any, Union

# Проверка строки на возможность быть float
def _is_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

    
# Выбор индекса value из expr, в случае отсутствия возвращается Default
def _get_index(expr: list, value: Any, default: Any):
    try:
        return expr.index(value)
    except ValueError:
        return default
    
# Разделение строки с выражением на список, в котором находятся числа (положительные
# и отрицательные), а также математические символы
def _split_expr(expr: Union[str, list]):
    if type(expr) == list:
        return expr
    expr = expr.replace(' ', '')
    expr_list = re.split('(\+|\-|\*|\/|\(|\))', expr)
    expr_list = list(filter(None, expr_list))
    minus_to_remove = []
    for i, val in enumerate(expr_list):
        if is_float(val):
            expr_list[i] = float(val)
        if val == '-' and (i == 0 or not is_float(expr_list[i-1])):
            expr_list[i+1] = '-'+expr_list[i+1]
            minus_to_remove.append(i)
    for index in sorted(minus_to_remove, reverse=True):
        del expr_list[index]
    return expr_list

# Поиск индекса с закрывающей скобкой
def _get_next_bracket_index(expr: list, index: int):
    opened_brackets = 1
    for i, val in enumerate(expr):
        if val == '(':
            opened_brackets+=1
        if val == ')':
            opened_brackets-=1
        if opened_brackets == 0:
            return i+index+1
        
        
def calc(expr: str, ndigits: int = 3):
    expr_list = _split_expr(expr)
    while '(' in expr_list:
        bracket_begin = expr_list.index('(')
        bracket_end = _get_next_bracket_index(expr_list[bracket_begin+1:], bracket_begin)
        
        expr_list = expr_list[:bracket_begin] + [calc(expr_list[bracket_begin+1:bracket_end])]+ \
                    expr_list[bracket_end+1:]
    
    # Данные знаки рассматриваются вместе, чтобы избежать ошибок типа 10/2*5 = 1 и 10*2/5 = 0 
    while '*' in expr_list or '/' in expr_list:
        sign_index = min(_get_index(expr_list, '*', len(expr_list)+1), 
                         _get_index(expr_list, '/', len(expr_list)+1))
        if expr_list[sign_index] == '*':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]*expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
        elif expr_list[sign_index] == '/':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]/expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
    
    while '+' in expr_list or '-' in expr_list:
        sign_index = min(_get_index(expr_list, '+', len(expr_list)+1), 
                         _get_index(expr_list, '-', len(expr_list)+1))
        
        if expr_list[sign_index] == '+':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]+expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
        elif expr_list[sign_index] == '-':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]-expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
        
    return round(expr_list[0], ndigits)


        
    